In [6]:
from pprint import pprint
from SimpleEarnAPY import main, formatDataFrame
from FetchBinance import Binance
assets = ['USDC', 'USDT']
binance = Binance(assets=assets)
rate = binance.simpleEarn('USDT')
rate = binance._GetCleanResult(rate)
pprint(rate)

{'USDT': {'latestAnnualPercentageRate': 0.04821731}}


In [ ]:
rate = binance.SimpleEarnRates
rate
df = formatDataFrame(binance.__class__.__name__, rate)
df

In [ ]:
for key, value in rate.items():
    print(key)
    print(value)

In [21]:
import pandas as pd

for asset in assets:
    df = pd.DataFrame.from_dict(rate[asset])
    df['coin'] = [asset]*len(df)
    df['cex'] = [binance.__class__.__name__]*len(df)
    df = df[['cex','coin','rate','amt']]

In [4]:
from SimpleEarnAPY import main
from pprint import pprint
df = main()

In [5]:
df

,cex,coin,rate,amt
0,Binance,USDC,15.8%,0-500USDC
1,Binance,USDC,6.8%,
2,Binance,USDT,10.0%,0-500USDT
3,Binance,USDT,5.0%,
4,Bitget,USDC,8.0%,0-100
5,Bitget,USDC,0.8%,"100-5,000"
6,Bitget,USDC,0.3%,"5,000-1,000,000"
7,Bitget,USDT,13.3%,0-500
8,Bitget,USDT,5.3%,"500-50,000,000"
9,OKX,USDC,7.0%,


In [7]:
print(df.to_string(index=False))

    cex coin  rate             amt
Binance USDC 15.8%       0-500USDC
Binance USDC  6.8%                
Binance USDT 10.0%       0-500USDT
Binance USDT  5.0%                
 Bitget USDC  8.0%           0-100
 Bitget USDC  0.8%       100-5,000
 Bitget USDC  0.3% 5,000-1,000,000
 Bitget USDT 13.3%           0-500
 Bitget USDT  5.3%  500-50,000,000
    OKX USDC  7.0%                
    OKX USDT  4.0%                
 Kucoin USDC  3.4%                
 Kucoin USDT  7.1%                
